In [ ]:
nx.graphviz_layout

In [13]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline


gg = nx.read_graphml('mnrasGraph.g', unicode)


In [18]:
%matplotlib inline
pos = nx.spring_layout(gg,scale=2)
nx.draw(gg,node_size=1,pos=nx.graphviz_layout(gg))
plt.show()


ImportError: pydot could not be loaded: http://code.google.com/p/pydot/

In [2]:
'''
This module allows you to create an animation of a graph with a 3D layout rotating in the 3D space about its centroid. The graph must be specified as an C{igraph.Graph} instance. You may specify an C{igraph.Layout} instance, or let the module compute a 3D C{'fr'} layout for you. You may optionally specify C{'color'} and C{'size'} attributes for the vertex sequence.

The output is a list of C{.png} snapshots of the rotating 3D view at 1 degree intervals. If you have C{ffmpeg} installed, all snapshots will be compiled into an C{.mp4} movie as well.
https://github.com/naviddianati/graph-anim
'''
import numpy.random as rd
#from mypalettes import mplPalette
import numpy as np
import igraph as ig
import os


def animate(g, layout = None, filename = "frame" , height = 800, rendermovie = True, outfile = 'anim'):
    """
    Rotate 3d layout of graph C{g} and export a C{.png} snapshot 
    for each angle.
    If found, the graphs vertex attributes C{size} and C{color}
    will be use. C{color} should be an RGB  tuple.
    If layout is None, a 3d C{FR} layout will be computed.

    @param g: an C{igraph.Graph} instance.
    @param layout: 3d igraph Layout. Optional.
    @param filename: filename prefix for C{.png} snapshots.
    @param height: height of output C{.png} files in pixels.
    @param rendermovie: whether or not to render a movie from the snapshots. Requires C{ffmpeg}.
    @param outfile: filename for output movie.
    """

    if 'size' not in g.vs.attribute_names():
        g.vs['size'] = g.degree() 
    
        
    if 'color' not in g.vs.attribute_names():
        g.vs['color'] = [(255,255,255)] * g.vcount()
    
    print "Framing the layout..."
    gg = frame_the_graph(g)

    if layout is None:
        print "computing 3d layout..."
        layout = g.layout_fruchterman_reingold_3d()


    #center the layout
    layout.center(layout.centroid()) 

    y_max = max([item[1] for item in layout.coords])
    x_max = max([item[0] for item in layout.coords])
    y_min = min([item[1] for item in layout.coords])
    x_min = min([item[0] for item in layout.coords])
    k = 1.5

    x_width = max([np.sqrt(point[0] ** 2 + point[2] ** 2) for point in layout.coords])    
    aspect = k*(x_max-x_min) / (y_max-y_min)

    fixed_coords = [[x_width*k,y_max],[x_width*k,y_min],[-x_width*k,y_min],[-x_width*k,y_max]]

    for i in range(360):
        rootpoint = layout.coords[0]
        rootpoint = (0,0,-10)
        layout.rotate(1, 0, 2, origin=rootpoint)
        coords2d = [point[:2] for point in layout.coords]
        
        new_layout = ig.Layout( coords2d +  fixed_coords)
        update_depths(gg,layout)
        
        ig.plot(gg,
            filename + '-%s.png' % str(i).zfill(3),
            vertex_color = gg.vs['color-2'],
            vertex_size = gg.vs['size'],
            vertex_frame_color = gg.vs['color-1'],
            layout = new_layout,
            edge_color = gg.es['color-1'],
            background='black',
            bbox=(0,0,int(aspect*height),height)
           )
    if rendermovie:
        result = os.system(' ffmpeg -y -r 60 -qscale 2 -i %s-%%03d.png %s.mp4' % (filename,outfile))
        if result != 0:
            print "Eerror: could not render movie with ffmpeg."
    






def frame_the_graph(g):
    """
    Add 4 dummy nodes to the graph to fix the view frame.
    
    @return: augmneted graph.
    """
    gg = g.copy()
    gg.add_vertices(4)
    gg.vs[-5:]['gen']= [None] * 4
    v1,v2,v3,v4 =  gg.vs.indices[-4:]
    gg.vs[-4:]['dummy'] = True
    gg.vs[-4:]['color'] = [(0,0,0)] * 4
    #gg.add_edges(((v1,v2),(v2,v3),(v3,v4),(v4,v1)))
    
    return gg




def update_depths(g,layout):
    """
    After rotating the 3d layout, update the vertex and edge
    colors according to the the new depths (M{z} coordinates).

    @param g: an C{igrpah.Graph} instance.
    @param layout: igraph 3d layout.
    """
    zs = np.array(layout.coords + [[0,0,0]] * 4)[:,2]
    zs += abs(min(zs))
    zs /= max(zs)

    g.vs['color-1'] = [rgba_to_color_name(list(v['color'])+[zs[i] ** 2]) if v['color'] else 'red' for i,v in enumerate(g.vs)]
    g.vs['color-2'] = [rgba_to_color_name(list(v['color'])+[zs[i]**2*2]) if v['color'] else 'red' for i,v in enumerate(g.vs)]
    g.es['color-1'] = [(0,0,0) if g.vs[e.source]['dummy'] else g.vs[e.source]['color-1'] for e in g.es]
    g.es['color-2'] = [(0,0,0) if g.vs[e.source]['dummy'] else g.vs[e.source]['color-2'] for e in g.es]


def rgba_to_color_name(rgba):
    '''
    @param rgba: a 4-tuple of float values in C{[0,1]}
    @return: a string such as C{"rgba(100,20,220,0.3)"}
    '''
    return "rgba(" + ",".join([str(int(255 * rgba[0])), str(int(255 * rgba[1])), str(int(255 * rgba[2])), '%0.2f' % rgba[3]]) + ")"




In [3]:
if __name__ == "__main__":
  #  g = ig.Graph.Barabasi(400,1)
    g = ig.read('mnrasGraph.g',format="graphml")
    g

In [9]:
ig.summary(g)
g.degree()

IGRAPH UN-- 2053 130740 -- 
+ attr: id (v), name (v)


[2615,
 325,
 3443,
 947,
 790,
 108,
 401,
 38,
 744,
 565,
 92,
 1222,
 766,
 132,
 564,
 143,
 1435,
 4518,
 5478,
 574,
 2124,
 282,
 733,
 1129,
 457,
 555,
 567,
 876,
 42,
 116,
 2460,
 450,
 21,
 89,
 2260,
 7,
 67,
 742,
 688,
 9,
 203,
 962,
 11,
 257,
 1738,
 3,
 115,
 1455,
 758,
 258,
 1096,
 243,
 4,
 1220,
 7,
 3347,
 31,
 2629,
 33,
 1817,
 672,
 133,
 1420,
 561,
 585,
 4,
 343,
 80,
 695,
 3,
 2665,
 434,
 570,
 94,
 48,
 68,
 1238,
 648,
 306,
 38,
 1908,
 52,
 998,
 621,
 2812,
 474,
 4,
 179,
 2864,
 595,
 1629,
 975,
 1679,
 609,
 1801,
 2397,
 913,
 1986,
 615,
 3043,
 13,
 516,
 2449,
 146,
 227,
 175,
 1265,
 865,
 314,
 145,
 1335,
 111,
 768,
 13,
 379,
 230,
 27,
 27,
 609,
 58,
 184,
 65,
 288,
 256,
 1264,
 27,
 161,
 133,
 252,
 724,
 230,
 200,
 96,
 14,
 60,
 2131,
 1451,
 239,
 906,
 3,
 18,
 1809,
 2418,
 6,
 440,
 509,
 184,
 1776,
 1258,
 45,
 303,
 103,
 349,
 2,
 954,
 700,
 687,
 559,
 1448,
 472,
 106,
 2118,
 2,
 22,
 800,
 31,
 649,
 817,
 4,


In [ ]:
  pal = mplPalette(max(g.degree()) + 1,'autumn')

    g.vs['color'] = [pal.get(d) for d in g.degree() ]
    

In [15]:
animate(g, filename = 'frame')

Framing the layout...
computing 3d layout...


KeyboardInterrupt: 